### Data Collection and Processing

In [1]:
# loading the csv data to a Pandas DataFrame
import pandas as pd
heart_data = pd.read_csv('heart.csv')

In [2]:
# print first 5 rows of the dataset
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [4]:
# number of rows and columns in the dataset
heart_data.shape

(303, 14)

In [5]:
# getting some info about the data
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
# checking for missing values
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [7]:
# checking the distribution of Target Variable
heart_data['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

1 --> Defective Heart

0 --> Healthy Heart

#### Splitting the Features and Target

In [8]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

### Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Define the features to be scaled
features_to_scale = X.columns.tolist()

# Fit the scaler to the data and transform the features
scaled_features = scaler.fit_transform(X[features_to_scale])

# Update the original DataFrame with the scaled features
X[features_to_scale] = scaled_features

### Splitting the Data into Training data & Test Data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [12]:
print(X.shape, X_train.shape, X_test.shape)

(303, 13) (242, 13) (61, 13)


### Evaluation

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
def evaluation(model):
    training_prediction = model.predict(X_train)
    test_prediction = model.predict(X_test)
    
    print("Accuracy Score of the model: ")
    
    training_data_accuracy = accuracy_score(Y_train, training_prediction)
    print("Accuracy score on training data: ", training_data_accuracy)

    test_data_accuracy = accuracy_score(Y_test, test_prediction)
    print("Accuracy score on test data: ", test_data_accuracy)
    
    print()
    print("Precision of the model: ")

    training_data_precision = precision_score(Y_train, training_prediction, average='binary')
    print("Precision score on training data: ", training_data_precision)

    test_data_precision = precision_score(Y_test, test_prediction, average='binary')
    print ("Precision score on test data: ", test_data_precision)
    print()
    print("Recall of the model: ")

    training_data_recall = recall_score(Y_train, training_prediction, average='binary')
    print("Recall score on training data: ", training_data_recall)

    test_data_recall = recall_score(Y_test, test_prediction, average='binary')
    print ("Recall score on test data: ", test_data_recall)
    print()
    print("f1 score of the model: ")

    training_data_f1 = f1_score(Y_train, training_prediction, average='binary')
    print("f1 score on training data: ", training_data_f1)

    test_data_f1 = f1_score(Y_test, test_prediction, average='binary')
    print ("f1 score on test data: ", test_data_f1)
    
    conf_matrix_train = confusion_matrix(Y_train, training_prediction)
    print()
    print("Confusion Matrix of Training Data: ")
    print(conf_matrix_train)

    print()
    print("Confusion Matrix of Test Data: ")
    conf_matrix_test = confusion_matrix(Y_test, test_prediction)
    print(conf_matrix_test)

### Model Training

In [14]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Define base classifiers
xgb = XGBClassifier()
knn = KNeighborsClassifier()
svc = SVC()

# Define StackingClassifier
estimators = [('xgb', xgb), ('knn', knn), ('svc', svc)]
stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=svc, cv=5)

# Train StackingClassifier
stacking_classifier.fit(X_train, Y_train)

StackingClassifier(cv=5,
                   estimators=[('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...)),
                               ('knn', KNeighborsClassifier()),
                               ('svc', SVC())],
                   final_estimator=SVC())

In [15]:
evaluation(stacking_classifier)

Accuracy Score of the model: 
Accuracy score on training data:  0.9669421487603306
Accuracy score on test data:  0.8032786885245902

Precision of the model: 
Precision score on training data:  0.9428571428571428
Precision score on test data:  0.8

Recall of the model: 
Recall score on training data:  1.0
Recall score on test data:  0.8484848484848485

f1 score of the model: 
f1 score on training data:  0.9705882352941176
f1 score on test data:  0.823529411764706

Confusion Matrix of Training Data: 
[[102   8]
 [  0 132]]

Confusion Matrix of Test Data: 
[[21  7]
 [ 5 28]]


### Saving the trained model

In [16]:
import pickle

In [17]:
filename = 'heart_disease_model_new.sav'
pickle.dump(stacking_classifier, open(filename, 'wb'))